In [1]:
import cv2
import pyautogui
import numpy as np
from keras.models import load_model
import keyboard
import cv2 as cv 
import numpy as np
import matplotlib.pyplot as plt
import mss.tools
import time
import uuid
import keyboard
import pyautogui
import time
import mss.tools
import uuid
import keyboard
from pynput import mouse

In [2]:
def screenshot():
    with mss.mss() as sct:
        time.sleep(1)
        monitor_info = sct.monitors[0]
        top_margin = (monitor_info["height"] // 3) + 50
        monitor = {"top": monitor_info["top"] + top_margin, "left": monitor_info["left"] + 55, "width": monitor_info["width"] - 120, "height": (monitor_info["height"] * 2 // 3)-105}
        output = f"testimages/{str(uuid.uuid4())}.png".format(**monitor)
        sct_img = sct.grab(monitor)
        mss.tools.to_png(sct_img.rgb, sct_img.size, output=output)
    return output

In [3]:
model = load_model('model_v2_iris.h5')

In [5]:
img_orig = cv.imread("DLIngrid/q/3bd9faf6-2375-4d13-b484-b468c8ae9f02.png")
img_rgb = cv.cvtColor(img_orig, cv.COLOR_BGR2RGB)
reshaped_image = np.expand_dims(img_rgb, axis=0)

error: OpenCV(4.8.1) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [26]:
prediction = model.predict(reshaped_image)

1/1 [==============================] - 1s 839ms/step


In [27]:
action_index = np.argmax(prediction)
confidence = prediction[0][action_index]

In [28]:
action_index = np.argmax(prediction)
confidence = prediction[0][action_index]
print(action_index)
print(confidence)

1
0.8737693


In [ ]:
time.sleep(5)
while not keyboard.is_pressed("space"):
    #test = 
    image = screenshot()


    

In [9]:
import splitfolders
input_folder = './dlcombineddata'
output_folder = './combinedsplit'
 
#split the images into 80% training and 20% testing sets
splitfolders.ratio(input_folder, output_folder, seed=40, ratio=(0.8, 0.2))

Copying files: 2602 files [00:24, 104.89 files/s]


In [45]:
from tensorflow.keras.applications.resnet50 import ResNet50
model = ResNet50(include_top=False, weights="imagenet", input_shape=(512, 512, 3))


In [1]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow.keras.optimizers as optimizers
from keras.layers import Dense, GlobalAveragePooling2D
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Flatten
from keras.layers import Reshape

In [4]:
model = ResNet50V2(include_top=False, weights="imagenet", input_shape=(224, 224, 3))

In [5]:
for layer in model.layers:
    layer.trainable = False

In [6]:
x = model.output

x = Dropout(0.5)(x)
x = BatchNormalization()(x)
#x = Reshape((1, 1, -1))(x)
#x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(x)  # Example convolutional layer
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Flatten()(x)
#x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dense(64, activation='relu')(x)

classes = Dense(3, activation='softmax')(x) 


model = Model(inputs=model.input, outputs=classes)
optimizer = optimizers.Adam(learning_rate=0.01)


In [15]:
model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

In [10]:
datagen = ImageDataGenerator(rescale=1./255,
                                rotation_range=20,
                                width_shift_range=0.1,
                                height_shift_range=0.1,
                                zoom_range=0.1,
                                shear_range=0.2,
                                channel_shift_range=0.2,
                                brightness_range=(0.8, 1.2),
                                #contrast_range=(0.9, 1.1),
                                #saturation_range=(0.8, 1.2),
                                
                                #horizontal_flip=True
                                )

In [11]:
train_data = datagen.flow_from_directory("./combinedsplit/train", target_size=(224, 224), batch_size=120, class_mode="categorical")
val_data = datagen.flow_from_directory("./combinedsplit/val", target_size=(224, 224), batch_size=120, class_mode="categorical")

Found 2080 images belonging to 3 classes.
Found 522 images belonging to 3 classes.


In [16]:
model.fit(train_data, epochs=10, validation_data=val_data)

Epoch 1/10
18/18 [==============================] - 114s 6s/step - loss: 1.3832 - accuracy: 0.7697 - val_loss: 1.0130 - val_accuracy: 0.7414
Epoch 2/10
18/18 [==============================] - 93s 5s/step - loss: 0.7941 - accuracy: 0.8144 - val_loss: 0.5725 - val_accuracy: 0.8199
Epoch 3/10
18/18 [==============================] - 94s 5s/step - loss: 0.4251 - accuracy: 0.8452 - val_loss: 0.4429 - val_accuracy: 0.8180
Epoch 4/10
18/18 [==============================] - 94s 5s/step - loss: 0.3823 - accuracy: 0.8659 - val_loss: 0.4797 - val_accuracy: 0.8123
Epoch 5/10
18/18 [==============================] - 94s 5s/step - loss: 0.3663 - accuracy: 0.8572 - val_loss: 0.3934 - val_accuracy: 0.8372
Epoch 6/10
18/18 [==============================] - 93s 5s/step - loss: 0.3309 - accuracy: 0.8750 - val_loss: 0.3962 - val_accuracy: 0.8544
Epoch 7/10
18/18 [==============================] - 94s 5s/step - loss: 0.3237 - accuracy: 0.8673 - val_loss: 0.4048 - val_accuracy: 0.8372
Epoch 8/10
18/18 [=

In [88]:
model.save('goodmaybe.keras')

In [90]:
model.save('goodmaybe.h5')

c:\Users\ingri\anaconda3\envs\mss\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [177]:
modelsaved = load_model('goodmaybe.keras')

In [20]:
img = cv.imread("croppeddataingrid\d/0c89b0ea-6314-415f-a6fb-b9772f845f7b.png")
img_rgb = cv.cvtColor(img_orig, cv.COLOR_BGR2RGB)
img = cv2.resize(img_rgb, (224, 224))
img_resized = img.astype('float32') / 255.0 
reshaped_image = np.expand_dims(img, axis=0)

In [5]:
from tensorflow.keras.applications.resnet_v2 import preprocess_input, decode_predictions
from tensorflow.keras.utils import load_img

In [10]:
def preprocess_image(input_image):
    # Resize the image to match the input size of your model
    original = cv.imread(input_image)
    colored = cv.cvtColor(original, cv.COLOR_BGR2RGB)
    resize = cv2.resize(colored, (224, 224))
    # Normalize pixel values to be in the range [0, 1]
    normalized = resize / 255.0
    # Expand dimensions to match the input shape expected by the model
    dimension = np.expand_dims(normalized, axis=0)
    y = preprocess_input(dimension)
    return y

In [6]:
modelsaved = load_model('modelmoredata.keras')

In [33]:
hmm = modelsaved.predict(preprocess_image("combinedsplit/val/forward/0c79ecf3-f9bd-4fd2-b607-7fe86db079d2.png"))

1/1 [==============================] - 0s 109ms/step


In [39]:
action_index = np.argmax(hmm)
confidence = hmm[0][action_index]
print(action_index)
print(confidence)

0
0.7527576


In [35]:
hmm[0]

array([0.7527576 , 0.1014059 , 0.14583649], dtype=float32)